In [214]:
import cv2 as cv
import PIL
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [215]:
test_path = "C:\Laya\ProjectManas\\neuralnetwork\\test\\test"
train_path = "C:\Laya\ProjectManas\\neuralnetwork\\train\\train"


In [216]:
for folder in os.listdir(test_path):
    print(folder)

bart_simpson
charles_montgomery_burns
homer_simpson
krusty_the_clown
lisa_simpson
marge_simpson
milhouse_van_houten
moe_szyslak
ned_flanders
principal_skinner


In [217]:
class_num = [x for x in range(len(os.listdir(train_path)))]
folder_dict = dict(zip(class_num, os.listdir(train_path)))
print(folder_dict)

{0: 'bart_simpson', 1: 'charles_montgomery_burns', 2: 'homer_simpson', 3: 'krusty_the_clown', 4: 'lisa_simpson', 5: 'marge_simpson', 6: 'milhouse_van_houten', 7: 'moe_szyslak', 8: 'ned_flanders', 9: 'principal_skinner'}


In [218]:
training_image_set = []
training_image_label = []
for i in range(len(os.listdir(train_path))):
    current_folder = str(train_path + "\\" + folder_dict[i])
    for j in os.listdir(current_folder):
        img = cv.imread(os.path.join(current_folder,j))
        training_image_set.append(img.flatten()/255)
        training_image_label.append(i)



test_image_set = []
test_image_label = []
for i in range(len(os.listdir(test_path))):
    current_folder = str(test_path + "\\" + folder_dict[i])
    for j in os.listdir(current_folder):
        img = cv.imread(os.path.join(current_folder,j))
        test_image_set.append(img.flatten()/255)
        test_image_label.append(i)



In [219]:
training_image_set = np.asarray(training_image_set)
training_image_set = training_image_set.T
training_image_label = np.asarray(training_image_label)

In [220]:
test_image_set = np.asarray(test_image_set)
test_image_set = test_image_set.T
test_image_label = np.asarray(test_image_label)

In [221]:
test_image_set.shape

(2352, 2000)

In [222]:
n, m = training_image_set.shape
def init_params():
    W1 = np.random.rand(10, 2352) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = sigmoid(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * sigmoid_prime(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [223]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 100 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
            
    return W1, b1, W2, b2

In [224]:
W1, b1, W2, b2 = gradient_descent(training_image_set, training_image_label, 0.3, 20000)

Iteration:  0
[2 2 2 ... 2 2 2] [0 0 0 ... 9 9 9]
0.10103788920845316
Iteration:  100
[9 8 6 ... 4 3 4] [0 0 0 ... 9 9 9]
0.12929848693259974
Iteration:  200
[1 8 5 ... 4 0 9] [0 0 0 ... 9 9 9]
0.16531199199699886
Iteration:  300
[1 8 5 ... 2 1 9] [0 0 0 ... 9 9 9]
0.19944979367262725
Iteration:  400
[9 8 5 ... 2 1 1] [0 0 0 ... 9 9 9]
0.22045767162686009
Iteration:  500
[9 8 5 ... 2 1 1] [0 0 0 ... 9 9 9]
0.23883956483681382
Iteration:  600
[9 8 5 ... 2 1 1] [0 0 0 ... 9 9 9]
0.2490934100287608
Iteration:  700
[9 8 5 ... 4 1 1] [0 0 0 ... 9 9 9]
0.25797173940227586
Iteration:  800
[9 8 5 ... 4 1 1] [0 0 0 ... 9 9 9]
0.2642240840315118
Iteration:  900
[9 7 5 ... 4 1 1] [0 0 0 ... 9 9 9]
0.2678504439164687
Iteration:  1000
[9 7 5 ... 4 1 1] [0 0 0 ... 9 9 9]
0.2727272727272727
Iteration:  1100
[9 7 5 ... 4 1 1] [0 0 0 ... 9 9 9]
0.2784794297861698
Iteration:  1200
[9 7 5 ... 4 1 1] [0 0 0 ... 9 9 9]
0.28548205577091407
Iteration:  1300
[9 7 5 ... 4 1 1] [0 0 0 ... 9 9 9]
0.2896086032262

In [225]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = test_image_set[:, index, None]
    prediction = make_predictions(test_image_set[:, index, None], W1, b1, W2, b2)
    label = test_image_label[index]
    return prediction[0], label
    


In [226]:
count = 0 
for i in range(len(test_image_label)):
    prediction, label = test_prediction(i, W1, b1, W2, b2)
    if prediction == label:
        count += 1


accuracy = count/len(test_image_label) * 100
print(accuracy)

28.199999999999996


# Bonus Task

In [227]:
bonus_test_image_set = []
bonus_test_image_label = []
bonus_path = "C:\Laya\ProjectManas\\neuralnetwork\\bonus_test\\test"
for i in os.listdir(bonus_path):
    img = cv.imread(os.path.join(bonus_path,i))
    bonus_test_image_set.append(img.flatten()/255)
    bonus_test_image_label.append(i)


bonus_test_image_set = np.asarray(bonus_test_image_set)
bonus_test_image_set = bonus_test_image_set.T
bonus_test_image_label = np.asarray(bonus_test_image_label)

count = 0 
for i in range(len(bonus_test_image_label)):
    prediction, label = test_prediction(i, W1, b1, W2, b2)
    if prediction == label:
        count += 1


accuracy = count/len(bonus_test_image_label) * 100
print(accuracy, count)

30.0 3
